<div style="text-align: left;">
    <img src="https://raw.githubusercontent.com/JLX626/py-aiml-aal-portfolio-edition/main/images/Australian-Apparel-Limited-Logo.png" alt="Australian Apparel Ltd" width="100"/>
    <h2>Australian Apparel Ltd.<br/>Data Cleaning and Preprocessing</h2>
</div>

## Notebook Overview

In this notebook, I demonstrate the data cleaning and preprocessing steps for Australian Apparel Limited's Q4 2020 sales data. My goal was to prepare the raw data for in-depth analysis, ensuring its quality and consistency. This process showcases my skills in Python, pandas, and numpy for real-world data manipulation and analysis.

## Skills and Tools Demonstrated

- Python programming (pandas, numpy)
- Data cleaning and preprocessing
- Data quality assessment and validation
- Time series data handling
- Data normalization techniques

## Data Cleaning and Preprocessing

### 1. Initial Data Inspection

I began by loading the dataset and performing a comprehensive inspection to understand its structure and quality.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Load the dataset
def load_data(file_path):
    return pd.read_csv(file_path)

# Inspect the data
def inspect_data(df):
    print("First few rows:")
    print(df.head())
    
    print("\nDataset Info:")
    print(df.info())
    
    print("\nSummary Statistics:")
    print(df.describe())
    
    print("\nMissing Values:")
    print(df.isnull().sum())
    
    print(f"\nNumber of duplicate rows: {df.duplicated().sum()}")
    
    print("\nColumn Names:")
    print(df.columns)
    
    print("\nUnique values in categorical columns:")
    for col in df.select_dtypes(include=['object']).columns:
        print(f"\n{col}:")
        print(df[col].unique())

# Load the data
df = load_data('../data/raw/AusApparalSales4thQrt2020.csv')

# Inspect the data
inspect_data(df)

First few rows:
         Date        Time State     Group  Unit  Sales
0  1-Oct-2020     Morning    WA      Kids     8  20000
1  1-Oct-2020     Morning    WA       Men     8  20000
2  1-Oct-2020     Morning    WA     Women     4  10000
3  1-Oct-2020     Morning    WA   Seniors    15  37500
4  1-Oct-2020   Afternoon    WA      Kids     3   7500

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7560 entries, 0 to 7559
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    7560 non-null   object
 1   Time    7560 non-null   object
 2   State   7560 non-null   object
 3   Group   7560 non-null   object
 4   Unit    7560 non-null   int64 
 5   Sales   7560 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 354.5+ KB
None

Summary Statistics:
              Unit          Sales
count  7560.000000    7560.000000
mean     18.005423   45013.558201
std      12.901403   32253.506944
min       2.000000    5000.0000

Key findings from my initial inspection:
- The dataset contains 7,560 entries with 6 columns
- There are no missing values or duplicates, which is a good start
- Categorical columns: Date, Time, State, Group
- Numerical columns: Unit, Sales

I noticed that the dataset covers sales from October 1, 2020, to December 30, 2020. Sales range from $5,000 to $162,500, with a mean of $45,013.56, while units sold range from 2 to 65, with a mean of 18 units. Interestingly, all major Australian states and territories are present, except for the Australian Capital Territory (ACT).

### 2. Data Cleaning and Transformation

#### Date Column Conversion

Next, I converted the 'Date' column to a datetime format. This step is crucial for any time-based analysis I might want to do later.

In [2]:
# Convert 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Verify the conversion
print("Data type of 'Date' column:", df['Date'].dtype)
print("\nFirst few entries of 'Date' column:")
print(df['Date'].head())

# Additional verification
print("\nDate range:")
print("Earliest date:", df['Date'].min())
print("Latest date:", df['Date'].max())

Data type of 'Date' column: datetime64[ns]

First few entries of 'Date' column:
0   2020-10-01
1   2020-10-01
2   2020-10-01
3   2020-10-01
4   2020-10-01
Name: Date, dtype: datetime64[ns]

Date range:
Earliest date: 2020-10-01 00:00:00
Latest date: 2020-12-30 00:00:00


By converting the dates from strings (like "1-Oct-2020") to datetime objects, I've made it much easier to perform operations like grouping sales by week or month, or calculating the time between purchases.

#### State Data Verification

I then took a closer look at the 'State' column to verify our earlier observation about ACT.

In [3]:
# Check unique values in the 'State' column
unique_states = df['State'].unique()
print("Unique states in the dataset:", unique_states)

# Verify absence of ACT
if 'ACT' not in unique_states:
    print("ACT (Australian Capital Territory) is not present in the dataset.")

Unique states in the dataset: [' WA' ' NT' ' SA' ' VIC' ' QLD' ' NSW' ' TAS']
ACT (Australian Capital Territory) is not present in the dataset.


I confirmed that ACT is indeed absent from the dataset. This could be because:
1. AAL doesn't operate in ACT
2. There was an oversight in data collection for ACT
3. ACT data might be combined with another state's data (often New South Wales)

This is an important point to keep in mind for any state-based analysis I might do later.

#### Month Column Creation

To facilitate monthly trend analysis, I created a new 'Month' column derived from the 'Date' column.

In [4]:
# Create 'Month' column
df['Month'] = df['Date'].dt.to_period('M')

# Verify the new column
print("First few rows with new 'Month' column:")
print(df[['Date', 'Month']].head())

# Check unique months
unique_months = df['Month'].unique()
print("\nUnique months in the dataset:")
print(unique_months)

First few rows with new 'Month' column:
        Date    Month
0 2020-10-01  2020-10
1 2020-10-01  2020-10
2 2020-10-01  2020-10
3 2020-10-01  2020-10
4 2020-10-01  2020-10

Unique months in the dataset:
<PeriodArray>
['2020-10', '2020-11', '2020-12']
Length: 3, dtype: period[M]


This new column will allow me to easily aggregate data by month without losing the granularity of the original date data.

#### Consistent Formatting

I then ensured consistent formatting across all columns, particularly the categorical ones.

In [5]:
# Check for leading/trailing whitespaces in string columns
string_columns = df.select_dtypes(include=['object']).columns
for col in string_columns:
    if df[col].str.strip().ne(df[col]).any():
        print(f"Column '{col}' has leading/trailing whitespaces.")
    else:
        print(f"Column '{col}' has no leading/trailing whitespaces.")

# Check for inconsistent formatting in categorical variables
categorical_columns = ['State', 'Group', 'Time']
for col in categorical_columns:
    print(f"\nUnique values in {col}:")
    print(df[col].unique())

Column 'Time' has leading/trailing whitespaces.
Column 'State' has leading/trailing whitespaces.
Column 'Group' has leading/trailing whitespaces.

Unique values in State:
[' WA' ' NT' ' SA' ' VIC' ' QLD' ' NSW' ' TAS']

Unique values in Group:
[' Kids' ' Men' ' Women' ' Seniors']

Unique values in Time:
[' Morning' ' Afternoon' ' Evening']


I removed any leading or trailing whitespaces from the 'State', 'Group', and 'Time' columns. This step is crucial for accurate grouping and analysis later on. Inconsistent formatting (like extra spaces) can lead to the same category being treated as different categories.

#### Data Range Validation

Next, I validated the ranges for the numerical columns.

In [6]:
# Remove leading/trailing whitespaces from categorical columns
categorical_columns = ['State', 'Group', 'Time']
for col in categorical_columns:
    df[col] = df[col].str.strip()

# Re-check unique values after cleaning
for col in categorical_columns:
    print(f"\nUnique values in {col} after cleaning:")
    print(df[col].unique())

# Verify no more whitespace issues
for col in categorical_columns:
    if df[col].str.strip().ne(df[col]).any():
        print(f"Column '{col}' still has whitespace issues.")
    else:
        print(f"Column '{col}' is clean of whitespace issues.")


Unique values in State after cleaning:
['WA' 'NT' 'SA' 'VIC' 'QLD' 'NSW' 'TAS']

Unique values in Group after cleaning:
['Kids' 'Men' 'Women' 'Seniors']

Unique values in Time after cleaning:
['Morning' 'Afternoon' 'Evening']
Column 'State' is clean of whitespace issues.
Column 'Group' is clean of whitespace issues.
Column 'Time' is clean of whitespace issues.


I confirmed that all 'Unit' and 'Sales' values are within expected ranges. This step helps identify any potential outliers or data entry errors that might skew our analysis.

### 3. Data Normalization and Final Preprocessing

As a final step, I applied normalization to the 'Sales' and 'Unit' columns.

In [7]:
# Identify numerical columns
numerical_columns = df.select_dtypes(include=[np.number]).columns

# Validate data ranges for numerical columns
for col in numerical_columns:
    print(f"\nStatistics for {col}:")
    print(df[col].describe())
    
    # Check for negative values in columns where it doesn't make sense
    if col in ['Unit', 'Sales']:
        neg_count = (df[col] < 0).sum()
        if neg_count > 0:
            print(f"Warning: {neg_count} negative values found in {col}")
        else:
            print(f"No negative values found in {col}")


Statistics for Unit:
count    7560.000000
mean       18.005423
std        12.901403
min         2.000000
25%         8.000000
50%        14.000000
75%        26.000000
max        65.000000
Name: Unit, dtype: float64
No negative values found in Unit

Statistics for Sales:
count      7560.000000
mean      45013.558201
std       32253.506944
min        5000.000000
25%       20000.000000
50%       35000.000000
75%       65000.000000
max      162500.000000
Name: Sales, dtype: float64
No negative values found in Sales


In [8]:
# Check for potential outliers or incorrect data
print(df[['Sales', 'Unit']].describe())

# Simple normalization for 'Sales' and 'Unit' columns
df['Sales_Normalized'] = (df['Sales'] - df['Sales'].min()) / (df['Sales'].max() - df['Sales'].min())
df['Unit_Normalized'] = (df['Unit'] - df['Unit'].min()) / (df['Unit'].max() - df['Unit'].min())

print("\nFirst few rows with normalized columns:")
print(df[['Sales', 'Sales_Normalized', 'Unit', 'Unit_Normalized']].head())

print("\nSummary of normalized columns:")
print(df[['Sales_Normalized', 'Unit_Normalized']].describe())

               Sales         Unit
count    7560.000000  7560.000000
mean    45013.558201    18.005423
std     32253.506944    12.901403
min      5000.000000     2.000000
25%     20000.000000     8.000000
50%     35000.000000    14.000000
75%     65000.000000    26.000000
max    162500.000000    65.000000

First few rows with normalized columns:
   Sales  Sales_Normalized  Unit  Unit_Normalized
0  20000          0.095238     8         0.095238
1  20000          0.095238     8         0.095238
2  10000          0.031746     4         0.031746
3  37500          0.206349    15         0.206349
4   7500          0.015873     3         0.015873

Summary of normalized columns:
       Sales_Normalized  Unit_Normalized
count       7560.000000      7560.000000
mean           0.254054         0.254054
std            0.204784         0.204784
min            0.000000         0.000000
25%            0.095238         0.095238
50%            0.190476         0.190476
75%            0.380952         0.

I used min-max normalization to bring these columns to a common scale of 0 to 1. This step is particularly useful when we want to compare or combine features that are on different scales.

Key observations from the normalized data:
- The mean of normalized data is about 0.254, or 25.4% of the maximum values
- The median is 0.190, indicating a right-skewed distribution
- The standard deviation is 0.205, suggesting a moderate spread of values

Finally, I saved the cleaned and preprocessed dataset for future analysis.

In [9]:
# Save the cleaned dataset
cleaned_data = df
cleaned_data.to_csv('../data/processed/cleaned_sales_data.csv', index=False)
print("Cleaned data saved successfully.")

Cleaned data saved successfully.


## Key Takeaways

Through this process, I:
1. Demonstrated my ability to handle and clean real-world sales data
2. Implemented robust data preprocessing techniques, including date conversion, categorical data cleaning, and normalization
3. Prepared a high-quality dataset ready for advanced analytics and visualization
4. Identified potential areas for further investigation, such as the right-skewed sales distribution and the absence of ACT data

## Next Steps

With this cleaned and normalized dataset, I am now ready for more in-depth analysis, including:
- Analyzing sales trends by state and customer group
- Performing time series analysis of monthly sales patterns
- Investigating the right-skewed sales distribution we observed

To give a glimpse of potential analysis directions, I've included some preliminary data groupings below. These initial groupings set the stage for more comprehensive explorations in subsequent notebooks. The work done here in cleaning and preprocessing the data will be crucial in ensuring the accuracy and reliability of any future analyses.

In [10]:
# Group data by State
state_groups = cleaned_data.groupby('State')

# Display summary statistics for each state
print("Summary statistics by State:")
print(state_groups['Sales'].describe())

# Group data by Customer Group
customer_groups = cleaned_data.groupby('Group')

# Display summary statistics for each customer group
print("\nSummary statistics by Customer Group:")
print(customer_groups['Sales'].describe())

Summary statistics by State:
        count          mean           std      min      25%      50%  \
State                                                                  
NSW    1080.0  69416.666667  20626.651646  30000.0  52500.0  70000.0   
NT     1080.0  20907.407407   8961.907893   5000.0  15000.0  20000.0   
QLD    1080.0  30942.129630  13344.638002   7500.0  20000.0  30000.0   
SA     1080.0  54497.685185  17460.965183  25000.0  40000.0  52500.0   
TAS    1080.0  21074.074074   9024.684205   5000.0  15000.0  20000.0   
VIC    1080.0  97745.370370  26621.597092  50000.0  77500.0  95000.0   
WA     1080.0  20511.574074   9231.905897   5000.0  12500.0  20000.0   

            75%       max  
State                      
NSW     85000.0  112500.0  
NT      27500.0   37500.0  
QLD     40000.0   62500.0  
SA      67500.0   87500.0  
TAS     27500.0   37500.0  
VIC    112500.0  162500.0  
WA      27500.0   37500.0  

Summary statistics by Customer Group:
          count          mean  

In [11]:
# Ensure 'Date' is in datetime format
cleaned_data['Date'] = pd.to_datetime(cleaned_data['Date'])

# Create a 'Month' column
cleaned_data['Month'] = cleaned_data['Date'].dt.to_period('M')

# Group by Month and calculate monthly summaries
monthly_sales = cleaned_data.groupby('Month').agg({
    'Sales': ['sum', 'mean', 'count'],
    'Unit': ['sum', 'mean']
})

# Rename columns for clarity
monthly_sales.columns = ['Total_Sales', 'Avg_Daily_Sales', 'Num_Transactions', 'Total_Units', 'Avg_Daily_Units']

# Reset index to make 'Month' a column
monthly_sales = monthly_sales.reset_index()

# Display the monthly summaries
print("Monthly Sales Summaries:")
print(monthly_sales)

# Calculate month-over-month growth rate for Total_Sales
monthly_sales['Sales_Growth_Rate'] = monthly_sales['Total_Sales'].pct_change() * 100

# Display the updated monthly summaries with growth rate
print("\nMonthly Sales Summaries with Growth Rate:")
print(monthly_sales)

Monthly Sales Summaries:
     Month  Total_Sales  Avg_Daily_Sales  Num_Transactions  Total_Units  \
0  2020-10    114290000     45353.174603              2520        45716   
1  2020-11     90682500     35985.119048              2520        36273   
2  2020-12    135330000     53702.380952              2520        54132   

   Avg_Daily_Units  
0        18.141270  
1        14.394048  
2        21.480952  

Monthly Sales Summaries with Growth Rate:
     Month  Total_Sales  Avg_Daily_Sales  Num_Transactions  Total_Units  \
0  2020-10    114290000     45353.174603              2520        45716   
1  2020-11     90682500     35985.119048              2520        36273   
2  2020-12    135330000     53702.380952              2520        54132   

   Avg_Daily_Units  Sales_Growth_Rate  
0        18.141270                NaN  
1        14.394048         -20.655788  
2        21.480952          49.234968  
